In [2]:
import xmlrpc.client
import json
import base64
import pandas as pd
import numpy as np
import re
#from office365.sharepoint.client_context import ClientContext
#from office365.sharepoint.files.file import File
#from office365.runtime.auth.user_credential import UserCredential
from pathlib import Path
import openpyxl 
from openpyxl.utils.cell import range_boundaries
import io
from openpyxl.utils.cell import coordinate_to_tuple
from openpyxl.utils import get_column_letter
from conn_sharepoint import get_file_from_sharepoint, get_auth_token, upload_file_to_sharepoint
from reportlab.lib.pagesizes import letter, A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak
from reportlab.platypus.frames import Frame
from reportlab.platypus.doctemplate import PageTemplate, BaseDocTemplate
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch, cm
from reportlab.lib import colors
from reportlab.lib.enums import TA_CENTER, TA_JUSTIFY, TA_LEFT, TA_RIGHT
from reportlab.platypus.flowables import HRFlowable
from datetime import datetime
from datetime import date
from datetime import timezone, timedelta
import load_dotenv
import os
import requests
import time
import pytz
import locale
import tabulate
import traceback
import certifi
import sqlite3



In [3]:
class Sharepoint:

    def __init__(self):
        self.token = get_auth_token()


        
    def download_file(self, file_name, folder_name = ''):
        """
        Descarga un archivo desde una carpeta específica de SharePoint.

        Args:
            file_name (str): Nombre del archivo a descargar.
            folder_name (str): Nombre de la carpeta dentro de la biblioteca de documentos.

        Returns:
            bytes: El contenido del archivo en formato binario si la descarga es exitosa.
            None: Si ocurre un error durante la descarga.
        """

        try:
            # Obtiene la referencia al archivo usando la URL relativa
            file = get_file_from_sharepoint(file_name, self.token)
            # Devuelve el contenido binario del archivo
            return file.content
        
        except Exception as e:
            # Si ocurre un error, lo muestra y devuelve None
            print(f"Error al descargar el archivo {file_name} de SharePoint: {e}")
            return None
        

    def upload_file(self, file_name, content_stream, retry_delay=10, max_retries=6, folder_name = ''):
        """Sube un archivo binario a una carpeta de SharePoint."""
        for attempt in range(max_retries):  # Intenta subir el archivo hasta max_retries veces
            try:
                # Obtiene la referencia de la carpeta destino en SharePoint
            
                content_stream.seek(0)  # Asegura que el stream esté al inicio
                # Sube el archivo usando el contenido del stream
                target_file = upload_file_to_sharepoint(file_name, self.token, content_stream.getvalue())

                # Ejecuta la consulta para completar la subida
                

                print(f"\n-> Archivo '{file_name}' subido con éxito en el intento {attempt + 1}.")
                return True  # Retorna True si la subida fue exitosa
            
            except Exception as e:
                error_str = str(e)
                # Verifica si el error es por archivo bloqueado en SharePoint
                if "SPFileLockException" in error_str or "423 Client Error: Locked" in error_str:
                    print(f"Intento {attempt + 1} fallido: el archivo '{file_name}' está bloqueado. Reintentando...")
                    if attempt < max_retries - 1:
                        time.sleep(retry_delay)  # Espera antes de reintentar
                    else:
                        print(f"No se pudo subir el archivo después de {max_retries} intentos. Por favor, asegúrate de que el archivo no esté en uso.")
                        return False  # Retorna False si no se pudo subir tras varios intentos
                else:
                    # Si es otro tipo de error, lo reporta y termina
                    print(f"Error al subir el archivo a SharePoint: {error_str}")
                    return None  # Retorna None en caso de error inesperado

        
sp = Sharepoint()

#print(sp.download_file('https://graph.microsoft.com/v1.0/drives/b!dx9RXh45RU6gEd39TWLgKItDBbzJweRPoWAkjonKJ4GcIDolNOD0TI7SvyLL7Hda/root:/04.%20Instalación%20y%20Mantenimiento/Trazabilidad%20de%20mantenciones%20y%20calibraciones/Captura.xlsx:/content'))

In [9]:
def modify_excel_file(resumen = '', sheet_name = '', table_name = ''):
    
    # Descarga el archivo Excel desde SharePoint
    excel = sp.download_file('https://graph.microsoft.com/v1.0/drives/b!dx9RXh45RU6gEd39TWLgKItDBbzJweRPoWAkjonKJ4GcIDolNOD0TI7SvyLL7Hda/root:/04.%20Instalación%20y%20Mantenimiento/Trazabilidad%20de%20mantenciones%20y%20calibraciones/Captura.xlsx:/content')
    #print(excel)
    if excel:
        try:
            # Convierte los bytes descargados en un objeto BytesIO para manipulación en memoria
            excel_file = io.BytesIO(excel)
            # Carga el archivo Excel en openpyxl
            wl = openpyxl.load_workbook(excel_file)

            # Selecciona la hoja de trabajo especificada
            wh = wl[sheet_name]

            # Obtiene la tabla de la hoja por su nombre
            tabla = wh.tables[table_name]
            
            # Obtiene la referencia actual de la tabla (ejemplo: 'A1:H10')
            ref_actual = tabla.ref
            # Extrae la coordenada final de la tabla (ejemplo: 'H10')
            coordenada_final = ref_actual.split(':')[-1]
            # Convierte la coordenada final en número de fila y columna
            fila_final_actual, columna_final_num = coordinate_to_tuple(coordenada_final)
            # Calcula la fila donde se insertarán los nuevos datos
            fila_inicio_nuevos_datos = fila_final_actual + 1
            
            # # Inserta los nuevos datos fila por fila en la hoja
            for i, fila_nueva in enumerate(resumen):
                for j, valor in enumerate(fila_nueva):
                    wh.cell(row=fila_inicio_nuevos_datos + i, column=j + 1, value=valor)
                    print(valor, table_name)
                    

            # Actualiza la referencia de la tabla para incluir las nuevas filas
            fila_final_nueva = fila_final_actual + len(resumen)
            columna_final_letra = get_column_letter(columna_final_num)
            referencia_inicial = ref_actual.split(':')[0]
            nueva_referencia = f'{referencia_inicial}:{columna_final_letra}{fila_final_nueva}'
            tabla.ref = nueva_referencia
            # --- Fin del código de openpyxl ---

            # # Guarda el archivo modificado en un nuevo stream de bytes
            excel_stream_out = io.BytesIO()
            wl.save(excel_stream_out)
            excel_stream_out.seek(0)  # Mueve el cursor al inicio del stream

            wb_check = openpyxl.load_workbook(excel_stream_out)
            ws_check = wb_check[sheet_name]
            ref_final = ws_check.tables[table_name].ref
            min_col, min_row, max_col, max_row = range_boundaries(ref_final)  # OJO: devuelve (min_col_idx, min_row, max_col_idx, max_row)

            print(f"Tabla '{table_name}' rango final: {ref_final}")
            # Mostrar las últimas N filas (por ejemplo, 5)
            N = 5
            start_row = max(min_row, max_row - N + 1)
            for r in range(start_row, max_row + 1):
                fila = [ws_check.cell(row=r, column=c).value for c in range(min_col, max_col + 1)]
                print(f"{r}: {fila}")
            
            excel_stream_out.seek(0)  # Asegura que el stream esté al inicio antes de subir
            #Sube el archivo modificado de vuelta a SharePoint
            success = sp.upload_file('https://graph.microsoft.com/v1.0/drives/b!dx9RXh45RU6gEd39TWLgKItDBbzJweRPoWAkjonKJ4GcIDolNOD0TI7SvyLL7Hda/root:/04.%20Instalación%20y%20Mantenimiento/Trazabilidad%20de%20mantenciones%20y%20calibraciones/Captura.xlsx:/content', excel_stream_out)

        except Exception as e:
            print(f"Error al procesar el archivo Excel: {e}")
            
    else:
        print("No se pudo descargar el archivo.")


p =[ [1,2], [3,4]]

modify_excel_file(p,'Mantenciones', 'resumen_mantenciones')

1 resumen_mantenciones
2 resumen_mantenciones
3 resumen_mantenciones
4 resumen_mantenciones
Tabla 'resumen_mantenciones' rango final: A1:K10
6: [50, 'Diego Marchant', '11/08/2025', 'Gestión Hídrica El Soldado', 'PM-4 Nodo B20', 'Sensor de Nivel', 'Keller PR-36XS 20mH2O', 1297053, 'MP', 'N° de serie no encontrado en Odoo. Revisar OT | informe_OT-50_2_MP_2.pdf', None]
7: [56, 'Diego Marchant', '12/08/2025', 'Gestión Hídrica El Soldado', 'PM-1 Nodo B17', 'Tablero', 'We nodo Lora y We PLC', 'Nodo 17', 'MP', 'N° de serie no encontrado en Odoo. Revisar OT | informe_OT-56_2_MP_1.pdf', None]
8: [56, 'Diego Marchant', '12/08/2025', 'Gestión Hídrica El Soldado', 'PM-1 Nodo B17', 'Sensor de Nivel', 'Keller PR-36XS 20mH2O', 1297008, 'MP', 'N° de serie no encontrado en Odoo. Revisar OT | informe_OT-56_2_MP_2.pdf', None]
9: [1, 2, None, None, None, None, None, None, None, None, None]
10: [3, 4, None, None, None, None, None, None, None, None, None]

-> Archivo 'https://graph.microsoft.com/v1.0/drives

In [4]:
df_capa_1

,OT,Técnico,Asset,Tipo de trabajo,Fecha visita,Cliente,Resolución visita,Calidad del Servicio
0,81,Elías Sanchez,[Gestión Hídrica El Soldado] Zona Veta Blanca ...,ST,2025-08-25 23:06:39,Superintendencia de recursos hídricos,"Se realiza visita para revisión del nivel, tod...",3
1,81,Elías Sanchez,[Gestión Hídrica El Soldado] Sector -300 Gateway,ST,2025-08-25 23:06:39,Superintendencia de recursos hídricos,Piscina UG-300 se regula la compuerta hacia e...,3
2,83,Diego Marchant,[Gestión hídrica El Soldado] Bodega S. Recurso...,ST,2025-08-27 00:09:31,Superintendencia de Recursos Hídricos.,Por fallido intento de trasladar bodega por pa...,3
3,87,David Loncopan,[Apr la peña] Oficina,C,2025-08-27 20:50:50,APR La Peña,Se realiza capacitación exitosa del personal d...,3
4,91,Elías Sanchez,[AA ES Charla de seguridad] Casino,C,2025-08-28 17:25:46,Superintendencia de recursos hídricos,Se realiza charla de seguridad por parte de AA...,3
...,...,...,...,...,...,...,...,...
101,176,Elías Sanchez,[Gestión Hídrica El Soldado] Pozo El Melón N° ...,ST,2025-10-07 01:03:48,Superintendencia de recursos hídricos,PEM-06: Se realiza inspección para verificar e...,5
102,176,Elías Sanchez,[Gestión Hídrica El Soldado] Pozo El Melón N° ...,ST,2025-10-07 01:03:48,Superintendencia de recursos hídricos,Se coordina con personal Anglo para visitar ca...,5
103,176,Elías Sanchez,[Gestión Hídrica El Soldado] Sector -300 Gateway,ST,2025-10-07 01:03:48,Superintendencia de recursos hídricos,Revisión de la instalación y su funcionamiento...,5
104,179,David Loncopan,[SSR Comité Manzanar] Lora Estanque Manzanar,ST,2025-10-07 18:24:49,Benjamin Castillo,Gateway lora en falla,5


In [5]:
with pd.ExcelWriter('resumen_trabajos.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    # Actualiza o agrega datos en la hoja 'Hoja1'
    df_capa_1.to_excel(writer, sheet_name='Capa_1', index=False)
    # Actualiza o agrega datos en la hoja 'Hoja2'
    df_capa_2.to_excel(writer, sheet_name='Capa_2', index=False)

In [ ]:
def modify_excel_file(resumen, sheet_name, table_name):
    # Descarga el archivo Excel desde SharePoint
    excel = sp.download_file('https://graph.microsoft.com/v1.0/drives/b!dx9RXh45RU6gEd39TWLgKItDBbzJweRPoWAkjonKJ4GcIDolNOD0TI7SvyLL7Hda/root:/04.%20Instalación%20y%20Mantenimiento/Trazabilidad%20de%20mantenciones%20y%20calibraciones/Captura.xlsx:/content')
    if excel:
        try:
            # Convierte los bytes descargados en un objeto BytesIO para manipulación en memoria
            excel_file = io.BytesIO(excel)
            # Carga el archivo Excel en openpyxl
            wl = openpyxl.load_workbook(excel_file)
            # Selecciona la hoja de trabajo especificada
            wh = wl[sheet_name]

            # Obtiene la tabla de la hoja por su nombre
            tabla = wh.tables[table_name]
            # Obtiene la referencia actual de la tabla (ejemplo: 'A1:H10')
            ref_actual = tabla.ref
            # Extrae la coordenada final de la tabla (ejemplo: 'H10')
            coordenada_final = ref_actual.split(':')[-1]
            # Convierte la coordenada final en número de fila y columna
            fila_final_actual, columna_final_num = coordinate_to_tuple(coordenada_final)
            # Calcula la fila donde se insertarán los nuevos datos
            fila_inicio_nuevos_datos = fila_final_actual + 1
                
            # Inserta los nuevos datos fila por fila en la hoja
            for i, fila_nueva in enumerate(resumen):
                for j, valor in enumerate(fila_nueva):
                    wh.cell(row=fila_inicio_nuevos_datos + i, column=j + 1, value=valor)
            
            # Actualiza la referencia de la tabla para incluir las nuevas filas
            fila_final_nueva = fila_final_actual + len(resumen)
            columna_final_letra = get_column_letter(columna_final_num)
            referencia_inicial = ref_actual.split(':')[0]
            nueva_referencia = f'{referencia_inicial}:{columna_final_letra}{fila_final_nueva}'
            tabla.ref = nueva_referencia
            # --- Fin del código de openpyxl ---

            # Guarda el archivo modificado en un nuevo stream de bytes
            excel_stream_out = io.BytesIO()
            wl.save(excel_stream_out)
            excel_stream_out.seek(0)  # Mueve el cursor al inicio del stream
            
            # Sube el archivo modificado de vuelta a SharePoint
            success = sp.upload_file('https://graph.microsoft.com/v1.0/drives/b!dx9RXh45RU6gEd39TWLgKItDBbzJweRPoWAkjonKJ4GcIDolNOD0TI7SvyLL7Hda/root:/04.%20Instalación%20y%20Mantenimiento/Trazabilidad%20de%20mantenciones%20y%20calibraciones/Captura.xlsx:/content', excel_stream_out)

        except Exception as e:
            print(f"Error al procesar el archivo Excel: {e}")
            
    else:
        print("No se pudo descargar el archivo.")




# Enviar datos a SharePoint
def send_data(df, sheet, table):
    manual = df.values.tolist()
    if manual != []:
        modify_excel_file(manual, sheet, table)

In [5]:

# #----------------------------------------------------------------------------------------
# #FUNCIONES

#Función para cargar archivo Excel
def modify_excel_file(resumen, sheet_name, table_name):
    # Inicializa la clase Sharepoint para manejar autenticación y operaciones
    # Descarga el archivo Excel desde SharePoint
    excel = sp.download_file('Captura.xlsx', '04. Instalación y Mantenimiento/Trazabilidad de mantenciones y calibraciones')
    if excel:
        try:
            # Convierte los bytes descargados en un objeto BytesIO para manipulación en memoria
            excel_file = io.BytesIO(excel)
            # Carga el archivo Excel en openpyxl
            wl = openpyxl.load_workbook(excel_file)
            # Selecciona la hoja de trabajo especificada
            wh = wl[sheet_name]

            # Obtiene la tabla de la hoja por su nombre
            tabla = wh.tables[table_name]
            # Obtiene la referencia actual de la tabla (ejemplo: 'A1:H10')
            ref_actual = tabla.ref
            # Extrae la coordenada final de la tabla (ejemplo: 'H10')
            coordenada_final = ref_actual.split(':')[-1]
            # Convierte la coordenada final en número de fila y columna
            fila_final_actual, columna_final_num = coordinate_to_tuple(coordenada_final)
            # Calcula la fila donde se insertarán los nuevos datos
            fila_inicio_nuevos_datos = fila_final_actual + 1
                
            # Inserta los nuevos datos fila por fila en la hoja
            for i, fila_nueva in enumerate(resumen):
                for j, valor in enumerate(fila_nueva):
                    wh.cell(row=fila_inicio_nuevos_datos + i, column=j + 1, value=valor)

            # Actualiza la referencia de la tabla para incluir las nuevas filas
            fila_final_nueva = fila_final_actual + len(resumen)
            columna_final_letra = get_column_letter(columna_final_num)
            referencia_inicial = ref_actual.split(':')[0]
            nueva_referencia = f'{referencia_inicial}:{columna_final_letra}{fila_final_nueva}'
            tabla.ref = nueva_referencia
            # --- Fin del código de openpyxl ---

            # Guarda el archivo modificado en un nuevo stream de bytes
            excel_stream_out = io.BytesIO()
            wl.save(excel_stream_out)
            excel_stream_out.seek(0)  # Mueve el cursor al inicio del stream
            
            # Sube el archivo modificado de vuelta a SharePoint
            success = sp.upload_file('Captura.xlsx', '04. Instalación y Mantenimiento/Trazabilidad de mantenciones y calibraciones', excel_stream_out)

        except Exception as e:
            print(f"Error al procesar el archivo Excel: {e}")
    else:
        print("No se pudo descargar el archivo.")




#Función para generar una lista de dataframes con los subs
def ordenar_respuestas(estructura, respuestas):
    #Mapeo de IDs de preguntas a títulos
    question_id_to_title = {q['questionId']: q['title'] for q in estructura['data']['questions']}

    #Lista de DataFrams para almacenar las respuestas
    dfs = []

    #Procesameinto de cada submission
    for submission in respuestas['data']['formSubmissions']: #A nivel de submission
        submission_data = {'#': submission['entryNum'], 'user': submission['submittingUserId']}  #Iniciar un diccionario con el ID de la submission
        #print(f"Submission ID: {submission['formSubmissionId']}")
        for answer in submission.get('answers', []): #A nivel de respuesta
            question_id = answer['questionId']
            question_title = question_id_to_title.get(question_id, f"Pregunta {question_id}") #Usamos el Id para buscar el título de la pregunta

            value = None    
            #Solo considerar aquellas respuestas que no estén vacías o ocultas
            if not answer.get('wasSubmittedEpmty', False) and not answer.get('wasHidden', False):
                question_type = answer.get('questionType', 'unknown')

                #Procesar según el tipo de pregunta
                if question_type == 'openEnded':
                    value = answer.get('value', 'error' )
                elif question_type == 'multipleChoice':
                    selected = [opt['text'] for opt in answer.get('selectedAnswers', [])]
                    value = ', '.join(selected) if selected else 'Ninguna respuesta seleccionada'
                elif question_type == 'yesNo':
                    value = answer.get('selectedIndex', '')
                elif question_type == 'datetime':
                    date_sub = answer.get('timestamp', '')
                    dt_utc = datetime.utcfromtimestamp(date_sub)
                    value = dt_utc
                elif question_type == 'description':
                    value = None
                elif question_type == 'image':
                    value = answer.get('images', '')
                elif question_type == 'signature':
                    value = answer.get('images', '')
                elif question_type == 'rating':
                    value = answer.get('ratingValue', '')
                else:
                    value = 'Tipo no reconocido'
            
            submission_data[question_title] = value
        
        df = pd.DataFrame([submission_data])  #Crear un DataFrame a partir de la respuesta
        dfs.append(df)  #Agregar el DataFrame a la lista
    return dfs
        
#Función para filtrar los subs por fecha        

def filter_submissions(API_key_connecteam):
    today = date.today()   

    #yesterday = today - timedelta(days=5)

    # Calcular el inicio del día (medianoche de hoy) en UTC
    start_of_day = datetime.combine(today, datetime.min.time(), tzinfo=timezone.utc)
    start_timestamp_ms = int(start_of_day.timestamp())

    # Calcular el fin del día (un milisegundo antes de la medianoche de mañana) en UTC
    end_of_day = datetime.combine(today + timedelta(days=1), datetime.min.time(), tzinfo=timezone.utc)
    end_timestamp_ms = int(end_of_day.timestamp()) - 1 # Restamos 1 ms para que sea el final de hoy

    #print(f"Start timestamp (ms): {start_timestamp_ms}")
    #print(f"End timestamp (ms): {end_timestamp_ms}")

    #print(yesterday)

    url = f"https://api.connecteam.com/forms/v1/forms/12914411/form-submissions?submittingStartTimestamp={start_timestamp_ms}&submittingEndTime={end_timestamp_ms}&limit=100&offset=0"

    headers = {
        "accept": "application/json",
        "X-API-KEY": f"{API_key_connecteam}"
    }

    response = requests.get(url, headers=headers)
    response_json = response.json()
    return response_json


def all_submission(API_key_connecteam):
    url = "https://api.connecteam.com/forms/v1/forms/12914411/form-submissions?limit=20&offset=0"

    headers = {"accept": "application/json",
            "X-API-KEY": f"{API_key_connecteam}"}

    response = requests.get(url, headers=headers)
    response_json = response.json()
    return response_json    


def form_structure(API_key_connecteam):
    url = "https://api.connecteam.com/forms/v1/forms/12914411"

    headers = {"accept": "application/json",
            "X-API-KEY": f"{API_key_connecteam}"}

    response = requests.get(url, headers=headers)
    response_json = response.json()
    return response_json

def user(API_key_connecteam, user_id):
    url = f"https://api.connecteam.com/users/v1/users?limit=10&offset=0&order=asc&userIds={int(user_id)}&userStatus=active"

    headers = {
        "accept": "application/json",
        "X-API-KEY": f"{API_key_connecteam}"
    }

    response = requests.get(url, headers=headers)

    response_json = response.json()

    nombre_usuario = response_json['data']['users'][0]["firstName"] + " " + response_json['data']['users'][0]["lastName"]
    return nombre_usuario





try:
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
except:
    pass  # Si no está disponible, usar el locale por defecto

def crear_estilos_personalizados():

    styles = getSampleStyleSheet()
    
    # Estilo para el título principal
    styles.add(ParagraphStyle(
        name='TituloPrincipal',
        parent=styles['Heading1'],
        fontSize=18,
        textColor=colors.black,
        spaceAfter=20,
        spaceBefore=10,
        alignment=TA_CENTER,
        fontName='Helvetica-Bold'
    ))
    
    # Estilo para subtítulos
    styles.add(ParagraphStyle(
        name='Subtitulo',
        parent=styles['Heading2'],
        fontSize=14,
        textColor=colors.black,
        spaceAfter=12,
        spaceBefore=16,
        fontName='Helvetica-Bold'
    ))
    
    # Estilo para información de fecha
    styles.add(ParagraphStyle(
        name='InfoFecha',
        parent=styles['Normal'],
        fontSize=10,
        textColor=colors.grey,
        alignment=TA_RIGHT,
        spaceAfter=20,
        fontName='Helvetica-Oblique'
    ))
    
    # Estilo para texto de introducción
    styles.add(ParagraphStyle(
        name='Introduccion',
        parent=styles['Normal'],
        fontSize=11,
        alignment=TA_JUSTIFY,
        spaceAfter=16,
        spaceBefore=8,
        leftIndent=0,
        rightIndent=0
    ))
    
    # Estilo para el pie de página
    styles.add(ParagraphStyle(
        name='PiePagina',
        parent=styles['Normal'],
        fontSize=8,
        textColor=colors.grey,
        alignment=TA_CENTER
    ))
    
    return styles

def crear_tabla_profesional(dataframe_trabajo, styles):

    campos = {
        'OT:': dataframe_trabajo.iloc[0,0],
        'Técnico:': dataframe_trabajo.iloc[0,1],
        'Proyecto:': dataframe_trabajo.iloc[0,2],
        'Fecha de realización:': dataframe_trabajo.iloc[0,3],
        'Cliente:': dataframe_trabajo.iloc[0,4],
        'Equipo/instrumento:': dataframe_trabajo.iloc[0,5],
        'Modelo:': dataframe_trabajo.iloc[0,6],
        'N° de serie:': dataframe_trabajo.iloc[0,7]

    }

    df_campos = pd.DataFrame(campos, index=[0])
    
    # Preparar datos para la tabla
    df_vertical = df_campos.T.reset_index()
    df_vertical.columns = ['Campo', 'Respuesta']
    
    # Crear encabezados de tabla
    headers = ['Campo', 'Respuesta']
    data_list = [headers]
    
    # Agregar datos con formato mejorado
    for _, row in df_vertical.iterrows():
        campo = str(row['Campo'])
        respuesta = str(row['Respuesta'])
        data_list.append([campo, respuesta])
    
    # Convertir a Paragraphs para mejor control del formato
    data_for_table = []
    for i, row in enumerate(data_list):
        if i == 0:  # Encabezados
            formatted_row = [Paragraph(f"<b>{str(cell)}</b>", styles['Normal']) for cell in row]
        else:  # Datos
            # Campo en negrita, respuesta normal
            campo_cell = Paragraph(f"<b>{str(row[0])}</b>", styles['Normal'])
            respuesta_cell = Paragraph(str(row[1]), styles['Normal'])
            formatted_row = [campo_cell, respuesta_cell]
        data_for_table.append(formatted_row)
    
    # Crear tabla con anchos optimizados
    table = Table(data_for_table, colWidths=[5*cm, 12*cm], repeatRows=1)
    

    table_style = TableStyle([
        # Encabezado
        ('BACKGROUND', (0, 0), (-1, 0), colors.orange),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 14),
        ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
        
        # Filas de datos
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
        ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 1), (-1, -1), 10),
        ('ALIGN', (0, 1), (0, -1), 'LEFT'),
        ('ALIGN', (1, 1), (1, -1), 'LEFT'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        
        # Bordes y espaciado
        ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
        ('TOPPADDING', (0, 0), (-1, -1), 8),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
        ('LEFTPADDING', (0, 0), (-1, -1), 12),
        ('RIGHTPADDING', (0, 0), (-1, -1), 12),
        
        # Alternar colores de fila para mejor legibilidad
        #('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.lightgrey])
    ])
    
    table.setStyle(table_style)
    return table

def informe_pdf_profesional(numero_visita, tipo_trabajo, dataframe_visita, dataframe_trabajo, equipo):
    id_tipo_mantención = {'MC': 'mantención correctiva',
                    'MP': 'mantención preventiva',
                    'I' : "instalación"}
    # Configuración del archivo
    pdf_file = f"informe_{tipo_trabajo}_OT-{dataframe_visita['#'][0]}_{equipo}.pdf"
    
    # Usar A4 para un aspecto más profesional
    doc = SimpleDocTemplate(
        pdf_file, 
        pagesize=A4,
        rightMargin=2*cm,
        leftMargin=2*cm,
        topMargin=2.5*cm,
        bottomMargin=2*cm
    )
    
    # Obtener estilos personalizados
    styles = crear_estilos_personalizados()
    
    # Lista de elementos del documento
    story = []
    
    # ENCABEZADO DEL DOCUMENTO
    # Título principal
    punto_monitoreo = dataframe_visita.get(f'{numero_visita}.1 Punto de monitoreo', ['N/A'])[0]
    title = Paragraph(
        f"Informe de trabajos<br/>{punto_monitoreo}",
        styles['TituloPrincipal']
    )
    story.append(title)
    
    # Información del nodo y número de visita
    # Línea divisoria decorativa
    line = HRFlowable(width="100%", thickness=1, lineCap='round', color=colors.orange)
    story.append(line)
    story.append(Spacer(1, 12))
    
    # Fecha de generación
    try:
        current_date = datetime.now().strftime("%d de %B de %Y")
    except:
        current_date = datetime.now().strftime("%d de %m de %Y")
    
    date_text = Paragraph(
        f"Fecha de generación: {current_date}",
        styles['InfoFecha']
    )
    story.append(date_text)
    #story.append(Spacer(1, 20))
    
    # SECCIÓN DE INTRODUCCIÓN
    intro_subtitle = Paragraph("Detalle", styles['Subtitulo'])
    story.append(intro_subtitle)
    
    ot_number = dataframe_visita['#'][0]


    intro_text = Paragraph(
        f"Este documento presenta el detalle de los trabajos de {id_tipo_mantención[tipo_trabajo]} "
        f"realizados en el marco de la Orden de Trabajo N° {ot_number}. ",
        styles['Introduccion']

    )

    story.append(intro_text)
    story.append(Spacer(1, 20))
    
    # Tabla principal con datos
    report_table = crear_tabla_profesional(dataframe_trabajo, styles)
    story.append(report_table)
    story.append(Spacer(1, 30))
    
    # SECCIÓN DE OBSERVACIONES (si hay observaciones específicas)
    if any('observacion' in str(col).lower() for col in dataframe_trabajo.columns):
        obs_subtitle = Paragraph("Observaciones Técnicas", styles['Subtitulo'])
        story.append(obs_subtitle)
        
        # Buscar columnas de observaciones
        obs_cols = [col for col in dataframe_trabajo.columns if 'observacion' in str(col).lower()]
        for col in obs_cols:
            if not dataframe_trabajo[col].isna().all():
                obs_text = str(dataframe_trabajo[col].iloc[0])
                if obs_text and obs_text.strip() and obs_text.lower() != 'nan':
                    obs_paragraph = Paragraph(
                        f"• {obs_text}",
                        styles['Introduccion']
                    )
                    story.append(obs_paragraph)
        story.append(Spacer(1, 20))
    
    # PIE DE PÁGINA INFORMATIVO
    story.append(Spacer(1, 30))
    line2 = HRFlowable(width="100%", thickness=0.5, lineCap='round', color=colors.lightgrey)
    story.append(line2)
    story.append(Spacer(1, 12))
    
    footer_text = Paragraph(
        f"Documento generado automáticamente • OT-{ot_number} • "
        f"{dataframe_trabajo.iloc[0,1]}",
        styles['PiePagina']
    )
    story.append(footer_text)
    
    # Construir el PDF
    doc.build(story)
    
    return pdf_file

# Función auxiliar para mantener compatibilidad con tu código existente
def informe_pdf(numero_visita, tipo_trabajo, dataframe_visita, dataframe_trabajo, equipo):
    return informe_pdf_profesional(numero_visita, tipo_trabajo, dataframe_visita, dataframe_trabajo, equipo)

# Resumen de notificaciones
def detalle_op(resumen, ot, tecnico, fecha, proyecto, punto, tipo, modelo, serial, trabajo_id, mensaje):
    resumen['OT'].append(ot)
    resumen['Técnico'].append(tecnico)
    resumen.setdefault('Fecha de revisión', []).append(fecha)
    resumen['Proyecto'].append(proyecto)
    resumen['Punto de monitoreo'].append(punto)
    resumen['Modelo'].append(modelo)
    resumen['N° serie'].append(serial)
    resumen['Tipo'].append(trabajo_id)
    resumen['Equipo/instrumento'].append(tipo)
    resumen['Mensaje'].append(mensaje)


#Provisorio
#data = pd.read_excel(r'C:\Users\DanielCuero\API_odoo\OT_borrador_2.xlsx')
#data = pd.read_excel(r'C:\Users\dacmx\API_Odoo\OT_borrador_2.xlsx')
#data = pd.read_excel(r'C:\Users\dacmx\API_Odoo\OT_b.xlsx')
#data = ordered_responses[0]  # Usar la primera OT de las respuestas ordenadas
#data_list = [data]



In [ ]:

#-----------------------------------------------------------------------------------------
#CONEXIÓN API 365
load_dotenv()
USERNAME = os.getenv('sharepoint_user')
PASSWORD = os.getenv('sharepoint_password')
SHAREPOINT_SITE = os.getenv('sharepoint_url_site')
SHAREPOINT_NAME_SITE = os.getenv('sharepoint_site_name')
SHAREPOINT_DOC = os.getenv('sharepoint_doc_library')

sp = Sharepoint()

#-----------------------------------------------------------------------------------------
#CONEXIÓN CON API CONNECTEAM

#Listas de subs

try:
    API_key_c = os.getenv('CONNECTEAM_API_KEY')
    ordered_responses = ordenar_respuestas(form_structure(API_key_c), filter_submissions(API_key_c))
except Exception as e:
    print(f"Ocurrio un problema con la conexión a la API-Connecteam: {e}")
#-----------------------------------------------------------------------------------------
#CONEXIÓN CON API ODOO
#Datos de conexión
os.environ['SSL_CERT_FILE'] = certifi.where()

url = os.getenv('URL_Odoo')
db = os.getenv('DB_Odoo')
username = os.getenv('USER_Odoo')
password = os.getenv('ODOO_API_KEY')

# #Inicio de sesión
try: 
    common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
    uid = common.authenticate(db, username, password, {})
except Exception as e:
    print(f"Error en el inicio de sesión desde la API: {e}")
    

#Inicialización del endpoint
try:
    models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))
except Exception as e:
    print(f"Error en la inicialización del endpoint: {e}")


#iNCLUIR OBSERVACIÓN COMO CAMPO DE BITÁCORA

# Se sebe además hacer la gestión de separar las líneas según el tipo de trabajo realizado, para llevaras a archivos distintos
resumen = {
    'OT': [],
    'Técnico': [],
    'Fecha de revisión': [],
    'Proyecto': [],
    'Punto de monitoreo': [],
    'Equipo/instrumento': [],
    'Modelo': [],
    'N° serie': [],
    'Tipo': [],
    'Mensaje': [],
    }


exito = {
    'OT': [],
    'Técnico': [],
    'Fecha de revisión': [],
    'Proyecto': [],
    'Punto de monitoreo': [],
    'Equipo/instrumento': [],
    'Modelo': [],
    'N° serie': [],
    'Tipo': [],
    'Mensaje': [],
    }


#-----------------------------------------------------------------------------------------
#PROCESAMIENTO DE INFORMACIÓN
#Analizando cada sub que esta en forma de dataframe

for df in ordered_responses:

    df = df.astype({'user': str})
    
    df_con_datos = df.dropna(axis=1, how ='all') #Eliminando las columnas que no se usaron
    df_columnas = df_con_datos.columns.to_list() #Lista de columnas que si tienen datos

    index_user = df_con_datos.columns.get_loc('user')

    try:
        user_name = user(API_key_c, df_con_datos['user'][0])
    except Exception as e:
        user_name = "Usuario no encontrado"
        print(f"Error al obtener el nombre del usuario: {e}")
        traceback.print_exc()
    
    try:
        df_con_datos.iloc[0, index_user] = user_name # Añadir el nombre del usuario al DataFrame
    except Exception as e:
        print(f"Error al asignar el nombre del usuario al DataFrame: {e}")
        traceback.print_exc()



    #Elementos globales
    id_tipo_de_trabajo = ['MP', 'MC', 'I']
    
    id_mantencion = {'MC': 'corrective',
                    'MP': 'preventive'}
    
    intalaciones_interes = ['Tablero', 'Caudalímetro', 'Sensor de nivel', 'Sonda multiparamétrica', 'Otro']

    #Puntos que efectivamente se visitaron
    numeros_visita = set()
    for col in df_columnas:
        # Verificamos si el nombre de la columna comienza con un dígito
        if col and col[0].isdigit():
            # Extraemos el primer carácter (el número)
            numeros_visita.add(col[0])

    numeros_visita = sorted(list(numeros_visita))
    #print(numeros_visita)
    #print(f"\nResultados de procesamiento | OT-{df_con_datos["#"][0]}: \n")
    for i in numeros_visita:

        #Separación de los trabajos realizados
        try: 
            tipos_realizados = [tipo.strip() for tipo in df_con_datos[f'{i}.2 Tipo de trabajo a realizar'][0].split(',') ]
        except:
            tipos_realizados = df_con_datos[f'{i}.2 Tipo de trabajo a realizar']

        # Columnas del punto {1} | general
        columnas_visita = [columna for columna in df_columnas if columna.startswith(i)]
        #columnas_visita.append(f'{i} Proyecto') 
        columnas_visita = ['#', 'user', 'Fecha visita ', 'Nombre del Cliente'] + columnas_visita 

        #Dejando un dataframe a nivel de visita de punto
        df_visita = df_con_datos[columnas_visita].copy()


        #Validando si el punto se encuentra seteadao en el listado de connecteam
        if df_visita[f'{i}.1 Punto de monitoreo'][0] == "No encontrado":
          
            #Creamos la columna proyecto
            try:
                index_columna_punto_proyecto = df_visita.columns.get_loc(f'{i} Proyecto')
                df_visita.loc[:, f"{i}.1 Proyecto"] = df_visita.iloc[0, index_columna_punto_proyecto]
                
                #Definimos el punto ingresado manaualmente como el verdadero
                index_columna_punto_no = df_visita.columns.get_loc(f'{i}.1 Punto de monitoreo')
                index_columna_punto_si = df_visita.columns.get_loc(f'{i}.1 Indicar nombre del punto')

                df_visita.iloc[0, index_columna_punto_no] = df_visita.iloc[0, index_columna_punto_si]

                del df_visita[f'{i}.1 Indicar nombre del punto']
            except Exception as e:
                print(f"Error al procesar el punto de monitoreo en OT {df_visita['#'][0]}: {e}")
                # Si no se encuentra la columna, asignamos un valor por defecto
                df_visita.loc[:, f"{i}.1 Proyecto"] = "Proyecto no especificado"
                df_visita.loc[:, f"{i}.1 Punto de monitoreo"] = "Punto no especificado"
                

        else:
            #Buscando el indice de columna
            index_columna_punto = df_visita.columns.get_loc(f'{i}.1 Punto de monitoreo')

            #Definiendo el nombre del proyecto
            try:
                match = re.search(r"\[([^\]]*)\]", df_visita.iloc[0, index_columna_punto])
                df_visita.loc[:, f"{i}.1 Proyecto"] = match.group(1)

            except Exception as e:
                continue
                            
            #Definiedo el nombre del punto
            df_visita.iloc[0, index_columna_punto] = re.sub(r"\[[^\]]*\]", "", df_visita.iloc[0, index_columna_punto]).strip() #Eliminando el nombre del proyecto

        #Definimos los ID de los tipos de trabajo realizados
        id_tipos_realizados = [item.split(' |')[0] for item in tipos_realizados]
        #id_tipos_realizados

        #Definimos los ID de tipos de trabajo de interes
        id_tipos_interes = []
        for tipo in id_tipos_realizados:
            if tipo in id_tipo_de_trabajo:
                id_tipos_interes.append(tipo)
        id_tipos_interes #[MC, MP]
        
        #Cantidad de MP realizadas
        MP_prefijo = set()
        for col in df_visita.columns:
            if ' MP |' in col: # Buscamos ' MP |' para identificar las columnas de MP
                # Extraemos el prefijo como '1.2.1 MP' o '1.2.2 MP'
                prefix_end_index = col.find(' MP |') + 4 # Sumamos 4 para incluir ' MP'
                prefix = col[:prefix_end_index].strip()
                MP_prefijo.add(prefix)
        
        conteo_instancias_MP = len(MP_prefijo)

        #Cantidad de MC realizadas
        MC_prefijo = set()
        for col in df_visita.columns:
            if ' MC |' in col: # Buscamos ' MC |' para identificar las columnas de MC
                # Extraemos el prefijo como '1.2.1 MC' o '1.2.2 MC'
                prefix_end_index = col.find(' MC |') + 4 # Sumamos 4 para incluir ' MC'
                prefix = col[:prefix_end_index].strip()
                MC_prefijo.add(prefix)
        
        conteo_instancias_MC = len(MC_prefijo)

        #Cantidad de I realizadas
        I_prefijo = set()
        for col in df_visita.columns:
            if ' I |' in col: # Buscamos ' MP |' para identificar las columnas de MP
                # Extraemos el prefijo como '1.2.1 MP' o '1.2.2 MP'
                I_prefix_end_index = col.find(' I |') + 4 # Sumamos 4 para incluir ' MP'
                I_prefix = col[:I_prefix_end_index].strip()
                I_prefijo.add(I_prefix)
        
        conteo_instancias_I = len(I_prefijo)
        #print(df_visita)


        for id in id_tipos_realizados:
            #Iniciamos la filtración por tipos de trabajo
            columnas_trabajo = [columna for columna in df_visita.columns if f'{id}' in columna]
            columnas_trabajo = ['#', 'user', f"{i}.1 Proyecto", 'Fecha visita ', 'Nombre del Cliente'] + columnas_trabajo
            df_trabajo = df_visita[columnas_trabajo]

            proyecto = df_visita[f"{i}.1 Proyecto"][0]
            punto = df_visita[f'{i}.1 Punto de monitoreo'][0]
            ot = df_visita['#'][0]
            fecha = df_visita['Fecha visita '][0]
            tecnico = df_visita['user'][0]


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_con_datos

,#,user,Fecha visita,Causa visita,Tipo de visita realizada:,1.1 Punto de monitoreo,1.2 Tipo de trabajo a realizar,1.2.1 I | Tipo de equipo/instrumento a instalar,1.2.1 I | Modelo,1.2.1 I | N° de serie,1.2.1 I | Observación,1.3 Resolución visita,1.4 Fotos recinto,¿Hubo retiro de residuos electrónicos o peligrosos defectuosos para gestión de correcta eliminación?,Nombre del Cliente,Firma del cliente,Calidad del Servicio
0,67,Elías Sanchez,2025-08-19 19:02:22,Pruebas con sonda multiparamétrica,"Visita ""Trabajos en terreno""",[Global Tórtolas] Pozo Dren ME Nodo,I | Instalación,Sonda multiparamétrica,HL4,21300H405140,No está calibrada,Se hacen pruebas para lectura a través de los ...,[{'url': 'https://public.cdn.connecteam.com/vw...,1,Patricio Cortez,[],3


Conexión con la API_Odoo

In [ ]:

#   CONEXIÓN CON Api
#Datos de conexión
load_dotenv.load_dotenv()
#CONEXIÓN CON API ODOO
#Datos de conexión
url = os.getenv('URL_Odoo')
db = os.getenv('DB_Odoo')
username = os.getenv('USER_Odoo')
password = os.getenv('ODOO_API_KEY')

os.environ['SSL_CERT_FILE'] = certifi.where()


# #Inicio de sesión
try: 
    common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
    uid = common.authenticate(db, username, password, {})

except Exception as e:
    print(f"Error en el inicio de sesión desde la API: {e}")
    
#Inicialización del endpoint
try:
    models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))
except Exception as e:
    print(f"Error en la inicialización del endpoint: {e}")

#------------------------------------------------------------------------
#CREACIÓN DE REQUEST
#Busqueda del ID del equipo en la base de datos maintenance.equipment
# serial = int(dic_trabajo['1.2.2 MC | N° de serie'])
# id_equipment = models.execute_kw(db, uid, password,
#     'maintenance.equipment', 'search',
#     [[
#         ['serial_no', '=', serial] #Bucamos el id de tabla del instrumento dentro de 'maintenance.equipment'
#     ]],
#     {'limit': 1})

# if id_equipment:
#     id_number = id_equipment[0]
#     print(f'ID numerico: {id_number}')


# #Crear un request para serial
# fields_values = {
#     'name': 'Prueba desde API II',
#     'equipment_id': id_number, #Aquí debemos usar el ID númerico de la sonda
#     'stage_id': '2', # 3 es finalizado 
#     'maintenance_type': id_mantencion['MC'],
#     'description': f'Última ubicación: {dic_trabajo[f'{'1'}.1 Punto de Monitoreo']}\nOT connecteam: {dic_trabajo['#']} | {dic_trabajo[f'{'1'}.2.2 MC | Observaciones']}'
# }

# created_request = models.execute_kw(db, uid, password,
#     'maintenance.request',
#     'create',
#     [fields_values]
#     )
# print(f'Request creada. ID: {created_request}')

#-------------------------------------------------------
#ACTULZACIÓN DE REGISTRO
#Busqueda del ID de del estado
# stage = 'Finalizado'
# id_stage = models.execute_kw(db, uid, password,
#     'maintenance.stage', 'search',
#     [[
#         ['name', '=', stage] #Bucamos el ID dentro de stage para "Finalizado"
#     ]],
#     {'limit': 1})

# stage_value = {
#     'stage_id': id_stage[0]
# }

# #Busqueda y cambio del request a actualizar
# #Busqueda de OT a actualizar
#
# OT_name = 'Prueba desde API II'
# OT_id = models.execute_kw(db, uid, password, #Esto nos entrega una lista
#     'maintenance.request', 'search',
#     [[
#         ['name', '=', OT_name] #Buscamos el ID de tabla de esta OT
#     ]],
#     {'limit': 1})

# if OT_id:
#     OT_number = OT_id[0] 


# #Con esto bsucamos el ID de la actividad existente para la OT_numbre
lead = models.execute_kw(db, uid, password,
    'crm.lead', 'search_read',
    [[
        ['name', '=', 'OT-00001'] #Buscamos el ID de tabla de esta OT
    ]],
    )




Revisión de leads

In [ ]:
#Autenticación con Odoo productivo
load_dotenv.load_dotenv()
#CONEXIÓN CON API ODOO
#Datos de conexión
url = os.getenv('URL_Odoo')
db = os.getenv('DB_Odoo')
username = os.getenv('USER_Odoo')
password = os.getenv('ODOO_API_KEY')

os.environ['SSL_CERT_FILE'] = certifi.where()

# #Inicio de sesión
try: 
    common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
    uid = common.authenticate(db, username, password, {})

except Exception as e:
    print(f"Error en el inicio de sesión desde la API: {e}")
    
#Inicialización del endpoint
try:
    models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))
except Exception as e:
    print(f"Error en la inicialización del endpoint: {e}")




In [ ]:
from datetime import datetime, time
from zoneinfo import ZoneInfo
# === Parámetros de filtrado ===
TARGET_STAGE_NAME = "Qualified"
LOCAL_TZ = ZoneInfo("America/Santiago")
LOCAL_DATE_STR = "2025-07-01"

# === Resolver etapa por nombre → id ===
stage = models.execute_kw(
    db, uid, password,
    "crm.stage", "search_read",
    [[("name", "=", TARGET_STAGE_NAME)]],
    {"limit": 1}
)

target_stage_id = stage[0]["id"]
target_stage_name = stage[0]["name"]


# === Construir fecha/hora de inicio (UTC) "desde el 01/07/2025 00:00:00 America/Santiago" ===
day_local = datetime.fromisoformat(LOCAL_DATE_STR)
start_local = datetime.combine(day_local.date(), time.min, tzinfo=LOCAL_TZ)
start_utc = start_local.astimezone(ZoneInfo("UTC")).replace(tzinfo=None)  # sin tz para Odoo
date_start_utc = start_utc.strftime("%Y-%m-%d %H:%M:%S")

field_ids = models.execute_kw(
    db, uid, password,
    "ir.model.fields", "search",
    [[("model", "=", "crm.lead"), ("name", "=", "stage_id")]],
    {"limit": 1}
)

domain = [
    ("field", "=", field_ids[0]),
    ("mail_message_id.model", "=", "crm.lead"),
    ("mail_message_id.date", ">=", date_start_utc),
    "|",
        ("new_value_integer", "=", target_stage_id),                          # hacia “Cotizado”
        "&", ("old_value_integer", "=", target_stage_id), ("new_value_integer", "!=", target_stage_id),  # desde “Cotizado”
]

tracking_ids = models.execute_kw(
    db, uid, password,
    "mail.tracking.value", "search",
    [domain],
    {"limit": 0}
)



In [ ]:
if tracking_ids:
    # 1️⃣ Leer los registros de seguimiento (tracking values)
    vals = models.execute_kw(
        db, uid, password,
        "mail.tracking.value", "read",
        [tracking_ids, [
            "mail_message_id",
            "new_value_integer", "old_value_integer",
            "new_value_char", "old_value_char"
        ]]
    )

    # 2️⃣ Obtener todos los IDs de mensajes relacionados
    message_ids = [v["mail_message_id"][0] for v in vals if v.get("mail_message_id")]

    # 3️⃣ Leer los mensajes para obtener fecha y autor
    messages = models.execute_kw(
        db, uid, password,
        "mail.message", "read",
        [message_ids, ["res_id", "date", "author_id"]]
    )

    # 4️⃣ Obtener los IDs de los leads
    lead_ids = sorted({m["res_id"] for m in messages if m.get("res_id")})
    leads_data = {}
    if lead_ids:
        leads = models.execute_kw(
            db, uid, password,
            "crm.lead", "read",
            [lead_ids, ["id", "name"]]
        )
        # Crear diccionario id → nombre
        leads_data = {l["id"]: l["name"] for l in leads}

    # 5️⃣ Cruzar la información de tracking + mensaje + nombre del lead
    msg_by_id = {m["id"]: m for m in messages}
    rows = []
    for v in vals:
        mid = v["mail_message_id"][0]
        msg = msg_by_id.get(mid, {})
        lead_id = msg.get("res_id")
        lead_name = leads_data.get(lead_id, "N/A")
        rows.append({
            "lead_name": lead_name,
            "change_date_utc": msg.get("date"),
            "author": msg.get("author_id")[1] if msg.get("author_id") else None,
            "old_stage": v.get("old_value_char"),
            "new_stage": v.get("new_value_char"),
        })

    # 6️⃣ Imprimir resultado legible
    print(f"Cambios a '{TARGET_STAGE_NAME}' encontrados: {len(rows)}\n")
    for r in rows:
        print(
            f"{r['lead_name']}: {r['old_stage']} → {r['new_stage']} | "
            f"{r['change_date_utc']} | {r['author']}"
        )
else:
    print("No se encontraron cambios de etapa en el rango especificado.")
x


Cambios a 'Qualified' encontrados: 68

WE2025-01250, Las Tórtolas, Cancha de salto: Prioridad semanal célula ad → Cotizado | 2025-07-01 16:47:32 | FELIPE QUEZADA GARCIA
WE2025-01320 Transmisión DGA: Prioridad semanal célula ad → Cotizado | 2025-07-14 13:08:33 | FELIPE QUEZADA GARCIA
WE2025-01322 Starlink, Puntos Riecillos: Prioridad semanal célula ad → Cotizado | 2025-07-14 13:36:42 | FELIPE QUEZADA GARCIA
WE2025-01294 Adicional sensor de nivel R2700: Prioridad semanal célula ad → Cotizado | 2025-07-14 13:37:12 | FELIPE QUEZADA GARCIA
WE2025-01294 Adicional sensor de nivel R2700: Cotizado → Alta Probabilidad de Adjudicación proximos 30 días | 2025-07-14 13:37:28 | FELIPE QUEZADA GARCIA
WE2025-01318 Licitación ENGIE: Cotizado → Rechazados | 2025-07-02 03:51:22 | FERNANDO FLORES PULGAR
WE2024-01152 - Nuevo Módulo de Logística: Generación de propuesta → Cotizado | 2025-07-02 12:57:21 | FELIPE QUEZADA GARCIA
WE2025-01250, Las Tórtolas, Cancha de salto: Cotizado → Alta Probabilidad de Adjud

In [49]:
df = pd.DataFrame(rows)

# Mostrar las primeras filas
print("\nVista previa del DataFrame:")

output_file = "cambios_etapa_cotizado.xlsx"

# Usamos el motor openpyxl para Excel moderno
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    df.to_excel(writer, index=False, sheet_name="Cambios Cotizado")



Vista previa del DataFrame:


In [ ]:
# Listado de puntos
#Listado con todos los equipos | Cada uno esta representado como un diccionario
equipos = models.execute_kw(db, uid, password,
    'maintenance.equipment', 'search_read',
    [[
        ['name', '!=', '']  # Buscar todos los equipos con un nombre no vacío
    ]],
    {'fields': ['id', 'name', 'serial_no', 'x_studio_location']}  # Limitar a 10 resultados para la prueba
    )

#Listado con todos los puntos de monitoreo indicando su id interno y nombre
puntos_odoo = models.execute_kw(db, uid, password,
    'x_maintenance_location', 'search_read',
    [[]],
    {'fields': ['id', 'x_name']}, 
    )




Listado de equipos

In [8]:


#EQUIPOS DENTRO DEL MÓDULO DE MANTENIMIENTO
#Obtención del ID de los equipos
try:
    id_equipos = models.execute_kw(db, uid, password, 
                'maintenance.equipment', 'search',
                [[]])

except Exception as e:
    print(e)

#Definición de los campos a solicitar
campos_equipo = ['name', 'serial_no', 'warranty_date', 
                 'x_studio_referencia_interna', 'x_studio_location', 'x_studio_ubicacin_1',  'x_studio_etwe_id', 
                  'x_studio_cuenta_analtica', 'x_studio_cliente_2', 
                 'x_studio_gua_de_despacho_1_filename']

#Solicitud de los datos de los equipos
try:
    request_data = models.execute_kw(db, uid, password,
        'maintenance.equipment', 'read',
        [id_equipos],
        {'fields': campos_equipo})
except Exception as e:
    print(e)

Equipos = {
    'N° de serie': [],
    'Nombre': [],
    'SKU': [],
    'Fecha de vencimiento garantía': [],
    'Cliente': [],
    'Punto de monitoreo': [],
    'id Punto': [],
    'Coordendas': [],
    'Cuenta analítica': [],
    'Guía de despacho': [],
}

#Iteración sobre los datos obtenidos
for e in request_data:
    #Queremos modificar el campo 'workcenter_id' dentro del diccionario

    location = False
    if e['x_studio_location'] is not False:
        location = re.sub(r"\[[^\]]*\]", "", e['x_studio_location'][1]).strip() #Eliminando el nombre del proyecto
        Equipos['Punto de monitoreo'].append(location)
    else:
        Equipos['Punto de monitoreo'].append(location)


    Equipos['N° de serie'].append(e['serial_no'])
    Equipos['Nombre'].append(e['name'])
    Equipos['SKU'].append(e['x_studio_referencia_interna'])
    Equipos['Fecha de vencimiento garantía'].append(e['warranty_date'])
    Equipos['Cliente'].append(e['x_studio_cliente_2'])
    Equipos['id Punto'].append(e['x_studio_etwe_id'])
    Equipos['Coordendas'].append(e['x_studio_ubicacin_1'])
    Equipos['Cuenta analítica'].append(e['x_studio_cuenta_analtica'])
    Equipos['Guía de despacho'].append(e['x_studio_gua_de_despacho_1_filename'])


df_equipos = pd.DataFrame(Equipos)

df_equipos.to_excel("dd.xlsx", index=False)

    

Listado de mantenimientos

In [1]:
#MATENIMIETOS DENTRO DEL MÓDULO DE MANTENIMIENTO

import xmlrpc.client
import pandas as pd
from dotenv import load_dotenv
import os
import re
import certifi
import ssl


os.environ['SSL_CERT_FILE'] = certifi.where()

load_dotenv()
url = os.getenv('URL_Odoo')
db = os.getenv('DB_Odoo')
username = os.getenv('USER_Odoo')
password = os.getenv('ODOO_API_KEY')

#Inicio de sesión
common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))

uid = common.authenticate(db, username, password, {})
common.version()

print("UDI:", uid)

#Inicialización del endpoint
models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))

#EQUIPOS DENTRO DEL MÓDULO DE MANTENIMIENTO
#Obtención del ID de los equipos
try:
    id_mantencion = models.execute_kw(db, uid, password, 
                'maintenance.request', 'search',
                [[]])

except Exception as e:
    print(e)


#Definición de los campos a solicitar
campos_mantencion = ['name', 'stage_id', 'maintenance_type', 'equipment_id', 'request_date', 'schedule_date','close_date']
try:
    request_data = models.execute_kw(db, uid, password,
        'maintenance.request', 'read',
        [id_mantencion],
        {'fields': campos_mantencion})
except Exception as e:
    print(e)


Mantenciones = {
    'Nombre': [],
    'Estado': [],
    'Tipo de mantenimiento': [],
    'Equipo': [],
    'N° de serie': [],
    'Fecha de solicitud': [],
    'Fecha programada': [],
    'Fecha de cierre': [],
}

#Iteración sobre los datos obtenidos
for e in request_data:
    #Queremos modificar el campo 'workcenter_id' dentro del diccionario
    if e['equipment_id'] is not False:
        Mantenciones['Equipo'].append(e['equipment_id'][1].split('/')[0])  
        Mantenciones['N° de serie'].append(e['equipment_id'][1].split('/')[-1])  
    else:
        Mantenciones['Equipo'].append(False)
        Mantenciones['N° de serie'].append(False)
    
    if e['stage_id'] is not False:
        Mantenciones['Estado'].append(e['stage_id'][1])
    else:
        Mantenciones['Estado'].append(False)


    Mantenciones['Nombre'].append(e['name'])
    Mantenciones['Tipo de mantenimiento'].append(e['maintenance_type'])
    Mantenciones['Fecha de solicitud'].append(e['request_date'])
    Mantenciones['Fecha programada'].append(e['schedule_date'])
    Mantenciones['Fecha de cierre'].append(e['close_date'])

df_mantenciones = pd.DataFrame(Mantenciones)
df_mantenciones

UDI: 82


,Nombre,Estado,Tipo de mantenimiento,Equipo,N° de serie,Fecha de solicitud,Fecha programada,Fecha de cierre
0,Mantenimiento preventivo - prueba,New Request,preventive,prueba,24000WE0001064,2025-12-23,2025-12-23 00:00:00,False
1,Preventive Maintenance - TABLERO SUTRON XLINK500,New Request,preventive,TABLERO SUTRON XLINK500,WE00000001516,2025-12-21,2025-12-21 00:00:00,False
2,Preventive Maintenance - prueba,New Request,corrective,prueba,24000WE0001064,2025-12-23,2025-12-23 00:00:00,False
3,Mantenimiento Preventivo - P3,New Request,preventive,False,False,2025-07-28,False,False
4,Mantenimiento Preventivo - P3G,New Request,preventive,False,False,2025-07-28,False,False


In [26]:
import sqlite3

#Eliminar un registro de la base de datos

with sqlite3.connect('form_entries.db') as connection:
    cursor = connection.cursor()
    cursor.execute("DELETE FROM processed_entries WHERE entry_id = ?", (67,))

    connection.commit()  # Asegura que los cambios se guarden

In [ ]:
asset_list = []
i = 1
while True:
    asset = []
    name = input(f'Nombre asset {i}: ')
    asset_id = input(f'ID asset {i}: ')
    coordenadas = input(f'Coordenadas (latitud, longitud) {i}: ')
    i += 1

    asset.append((name, asset_id, coordenadas))
    asset_list.append(asset)

    print('-------------------------------------------')
    continuar = input('¿Desea agregar otro asset? (s/n): ')
    if continuar.lower() != 's':
        break

-------------------------------------------
-------------------------------------------
-------------------------------------------
-------------------------------------------


Actualización de coordenadas de Asset

In [ ]:
import xmlrpc.client
import pandas as pd
from dotenv import load_dotenv
import os
import re
import certifi
import ssl


os.environ['SSL_CERT_FILE'] = certifi.where()

load_dotenv(r"C:\Users\DanielCuero\.env")
#load_dotenv(r"C:\Users\dacmx\.env")

url = os.getenv('URL_Odoo')
db = os.getenv('DB_Odoo')
username = os.getenv('USER_Odoo')
password = os.getenv('ODOO_API_KEY')

#Inicio de sesión
try: 
    common = xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(url))
    uid = common.authenticate(db, username, password, {})
except Exception as e:
    print(f"Error en el inicio de sesión desde la API: {e}")
    

#Inicialización del endpoint
try:
    models = xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(url))
except Exception as e:
    print(f"Error en la inicialización del endpoint: {e}")


ubicaciones = {
    1031: (-32.906859, -71.208933),
    1628: (-32.638493, -71.179153),
    1627: (-32.646053, -71.178101),
    1625: (-32.6515999, -71.130112),
    1603: (-33.268053, -70.678775),
    1535: (-33.112094, -70.748532),
    1489: (-23.3356644, -69.8379802),
    1484: (-33.12608, -70.76798),
    1483: (-33.1248, -70.76808),
    1482: (-33.12715, -70.76669),
    1481: (-33.10978, -70.74866),
    1480: (-33.10104, -70.75002),
    1458: (-33.276068, -70.668437),
    1457: (-33.426559, -70.803482),
    1456: (-22.888905, -68.173126),
    1452: (-33.427348, -70.771908),
    1451: (-41.426111, -72.955639),
    1444: (-34.200676, -71.5618248),
    1442: (-33.775296, -70.744946),
    1437: (-33.433778, -70.77975),
    1435: (-34.724183, -71.063982),
    1431: (-35.0479694, -71.2860007),
    1428: (-32.636687, -71.114803),
    1426: (-33.266627, -70.649707),
    1424: (-23.124969, -69.855088),
    1423: (-23.588253, -70.254692),
    1414: (-33.426559, -70.803482),
    1413: (-33.449014, -70.858906),
    1412: (-33.11095, -70.74781),
    1411: (-33.11115, -70.74822),
    1410: (-33.11208, -70.74879),
    1409: (-33.11193, -70.74914),
    1408: (-33.11224083, -70.74902777),
    1404: (-33.12930572, -70.76169238),
    1403: (-33.11596781, -70.71184977),
    1402: (-33.11635311, -70.71137499),
    1400: (-24.177299, -69.056289),
    1399: (-24.177299, -69.056289),
    1361: (-32.64084478, -71.17481578),
    1360: (-32.64034127, -71.17472898),
    1359: (-32.63992716, -71.17468698),
    1358: (-32.63951232, -71.1746876),
    1357: (-32.63897493, -71.17499434),
    1356: (-32.63892521, -71.17578186),
    1355: (-32.63856806, -71.17661522),
    1354: (-32.63806973, -71.1762301),
    1353: (-32.63763071, -71.1771041),
    1352: (-32.63738063, -71.17800399),
    1351: (-32.6385422, -71.18692117),
    1350: (-32.64116776, -71.17283043),
    1349: (-32.64116776, -71.17283043),
    1348: (-32.63716941, -71.17822269),
    1347: (-32.63804413, -71.17874483),
    1346: (-32.63937543, -71.17945926),
    1345: (-32.64119072, -71.18034533),
    1290: (-32.64194297, -71.17337146),
    1235: (-32.69706723, -71.20764124),
    1234: (-32.74978835, -71.19108794),
    1233: (-32.75209204, -71.18827365),
    1232: (-32.69805088, -71.21067308),
    1231: (-32.63716941, -71.17822269),
    1230: (-32.70258216, -71.20684916),
    1229: (-18.444472, -69.879444),
    1228: (-32.6461347, -71.177902),
    1217: (-32.6367046, -71.1184939),
    1212: (-33.3811302185059, -70.5914916992188),
    1208: (-33.06308, -70.37545),
    1207: (-33.03317, -70.3551),
    1205: (-32.659672, -71.154574),
    1204: (-33.374973, -70.59745),
    1203: (-32.6527535, -71.1578216),
    1202: (-20.967138, -69.682861),
    1199: (-32.68618, -71.19052),
    1197: (-34.4247589111328, -70.8697052001953),
    1179: (-32.6491538, -71.1912428),
    1178: (-32.8258872, -71.0591013),
    1177: (-32.660463, -71.16501),
    1176: (-32.661056, -71.164623),
    1175: (-32.665518, -71.170395),
    1174: (-32.665518, -71.170395),
    1173: (-32.665518, -71.170395),
    1172: (-32.652539, -71.164652),
    1171: (-32.654864, -71.143584),
    1169: (-33.0019918, -70.8864068),
    1168: (-32.8626967, -70.8878183),
    1167: (-32.8496725, -70.9141325),
    1165: (-32.750421, -70.558124),
    1164: (-33.0049753, -70.889036),
    1163: (-33.1751086, -70.6902815),
    1161: (-32.8966779, -71.3253643),
    1160: (-32.9151367, -71.3719724),
    1159: (-32.7590529, -70.97665),
    1158: (-32.6617007, -71.1996448),
    1156: (-32.7319801, -70.9270319),
    1155: (-32.689686, -71.2162958),
    1154: (-32.685511, -71.223232),
    1152: (-33.109373, -70.789594),
    1145: (-40.588497, -73.126686),
    1144: (-40.583176, -73.148007),
    1117: (-32.865613, -70.991819),
    1116: (-32.865306, -70.992222),
    1107: (-32.959235, -71.510531),
    1103: (-32.932205, -71.355972),
    1102: (-32.6601984, -71.1454712),
    1092: (-30.698586, -70.953128),
    1085: (-36.977379, -73.171247),
    1084: (-32.683702, -71.186713),
    1083: (-32.660456, -71.144889),
    1082: (-32.658837, -71.144007),
    1081: (-40.9201545715332, -73.1641693115234),
    1058: (-32.1174673, -71.4362696),
    1055: (-32.924067, -71.440254),
    1052: (-32.920845, -71.443065),
    1044: (-32.890226, -71.452407),
    1043: (-32.721444, -71.234167),
    1040: (-32.79353, -71.16331),
    1039: (-32.795653, -71.164821),
    1038: (-33.133586, -70.811163),
    1037: (-33.131596, -70.806992),
    1036: (-33.12925, -70.80225),
    1035: (-33.1408925, -70.7949074),
    1030: (-32.900806, -71.215477),
    1020: (-33.129849, -70.80481),
    1017: (-32.7275314331055, -71.2047958374023),
    1015: (-32.8255697, -71.0243006),
    1014: (-32.825937, -71.059669),
    1013: (-32.8114082, -71.0656414),
    1010: (-33.5958814, -70.6590241),
    1009: (-23.253378, -70.109236),
    1008: (-22.678643, -70.190429),
    1007: (-32.814773, -71.270217),
    1006: (-32.818019, -71.251373),
    1005: (-23.256353, -70.114731),
    1004: (-33.7583618164062, -71.3328399658203),
    1003: (-32.899047, -71.288502),
    993: (-32.890903, -71.3148789),
    992: (-32.7975387573242, -70.8866195678711),
    991: (-32.8843865, -71.0830038),
    990: (-32.8775882, -71.1033627),
    988: (-32.8843865, -71.0830038),
    981: (-32.7421989440918, -70.784538269043),
    980: (-32.783824, -70.853959),
    979: (-32.783596, -70.853561),
    978: (-32.899475, -71.222893),
    976: (-32.775152, -70.836726),
    975: (-32.769878, -70.834312),
    972: (-32.781052, -70.848488),
    971: (-33.171186, -70.6277),
    967: (-32.9168927, -71.3745282),
    966: (-32.8809427, -71.0978866),
    965: (-32.949292, -71.265644),
    964: (-32.939812, -71.267609),
    963: (-32.9168927, -71.3745282),
    962: (-32.8518021, -71.1808209),
    961: (-32.8430518, -71.2232792),
    960: (-32.7697052, -70.8007508),
    959: (-32.784971, -70.860113),
    958: (-32.781105, -70.848465),
    941: (-32.84974, -70.914508),
    940: (-32.854281, -70.91496),
    938: (-39.451509, -72.784093),
    936: (-32.683935, -71.212404),
    932: (-32.852472, -70.982389),
    931: (-34.637314, -71.13011),
    927: (-35.83084, -71.641721),
    923: (-32.663881, -71.167709),
    921: (-32.384153, -71.102681),
    918: (-32.8092772, -70.9614416),
    917: (-32.7674, -70.55631),
    916: (-32.767073, -70.554522),
    905: (-40.8563172, -73.1698335),
    904: (-32.660842, -71.217868),
    903: (-32.745186, -71.192663),
    902: (-32.69702, -71.207538),
    901: (-32.808308, -70.93372),
    900: (-32.801331, -70.905869),
    898: (-32.806642, -70.903452),
    897: (-32.697128, -71.207619),
    896: (-23.124722, -69.873611),
    887: (-32.802879, -70.898046),
    882: (-33.092399, -70.667076),
    881: (-33.349618, -70.306104),
    870: (-36.770618, -73.125965),
    861: (-33.10951, -70.74793),
    860: (-33.11095, -70.74781),
    859: (-33.11115, -70.74822),
    858: (-33.11208, -70.74879),
    857: (-33.11169, -70.74917),
    856: (-33.11193, -70.74914),
    855: (-33.11177, -70.7489),
    854: (-33.11167, -70.7489),
    853: (-33.11163, -70.74897),
    852: (-33.10481, -70.74995),
    851: (-33.10125, -70.75065),
    850: (-33.10183, -70.75242),
    848: (-33.09274, -70.72462),
    847: (-33.09363, -70.71398),
    846: (-33.08831, -70.73432),
    845: (-33.08356, -70.73114),
    844: (-33.15509, -70.6593),
    843: (-33.15475, -70.660973),
    842: (-33.13357, -70.68394),
    841: (-33.1354, -70.68243),
    840: (-33.15269, -70.68343),
    839: (-33.15687, -70.68707),
    838: (-33.15501, -70.66394),
    837: (-33.15507, -70.675),
    836: (-32.65876521, -71.1480329),
    835: (-32.84305, -70.526938),
    831: (-32.667411, -70.531416),
    830: (-32.663254, -70.534191),
    828: (-32.913348, -70.30821),
    827: (-33.280973, -70.652863),
    826: (-32.7674, -70.55631),
    816: (-33.307423, -70.685064),
    815: (-33.30423, -70.699193),
    814: (-33.272386, -70.695966),
    813: (-33.441813, -70.808936),
    812: (-33.424655, -70.792991),
    810: (-32.696407, -71.211547),
    808: (-32.695526, -71.212631),
    795: (-32.728897, -71.235518),
    790: (-32.662517, -71.200851),
    752: (-32.68335, -71.216565),
    750: (-33.442492, -70.814939),
    747: (-32.761776, -71.181862),
    746: (-32.77978741, -70.98014033),
    745: (-32.779389, -70.978278),
    705: (-38.9805488, -72.6346734),
    671: (-33.195253, -70.6496),
    662: (-32.892565, -70.377521),
    661: (-32.902792, -70.3686698),
    660: (-32.833667, -70.527111),
    659: (-32.835639, -70.531694),
    658: (-32.821417, -70.543),
    657: (-32.829472, -70.549806),
    656: (-32.784173, -70.5482),
    655: (-32.781131, -70.570312),
    654: (-32.7354316, -70.5639349),
    651: (-32.894694, -70.564333),
    650: (-32.886972, -70.5833778),
    649: (-32.8873165, -70.5858946),
    648: (-32.8875717, -70.5912306),
    647: (-32.88763, -70.594849),
    456: (-32.899839, -70.620016),
    455: (-32.889583, -70.61475),
    448: (-32.88575, -70.593),
    447: (-32.8725, -70.584833),
    446: (-32.6857735, -71.1901755),
    425: (-32.739546, -70.571249),
    424: (-33.415059, -70.575599),
    423: (-32.7640266, -70.5297444),
    422: (-32.7640266, -70.5297444),
    414: (-33.344624, -70.711113),
    413: (-23.611652, -70.387392),
    411: (-32.824389, -70.422667),
    410: (-32.824389, -70.422667),
    409: (-32.700369, -71.209374),
    408: (-32.692583, -70.927111),
    407: (-32.698056, -70.939806),
    406: (-32.779528, -70.878694),
    405: (-32.784833, -70.895639),
    404: (-32.745917, -70.9685),
    403: (-32.7455, -70.967306),
    402: (-32.748778, -70.963972),
    401: (-32.74875, -70.95525),
    400: (-32.750778, -70.956583),
    394: (-32.717306, -70.915861),
    393: (-32.716667, -70.917028),
    392: (-32.709889, -70.937861),
    379: (-32.817194, -70.976778),
    375: (-32.759111, -70.976722),
    374: (-32.765583, -70.969694),
    371: (-32.731917, -70.927083),
    370: (-32.729722, -70.937556),
    366: (-32.8591299, -70.4914983),
    365: (-32.859662, -70.491731),
    364: (-32.8555355, -70.5062355),
    362: (-32.85497, -70.506856),
    361: (-32.841894, -70.527797),
    360: (-32.8362432, -70.5436594),
    359: (-32.8371379, -70.5444066),
    358: (-32.835142, -70.546176),
    336: (-32.907269, -70.287918),
    334: (-32.862389, -70.41525),
    333: (-32.862722, -70.416111),
    332: (-32.695759, -71.209289),
    326: (-32.660417, -71.210611),
    318: (-32.693, -71.210917),
    316: (-32.696849, -71.1895726),
    315: (-32.6826655, -71.1893858),
    314: (-32.6844274, -71.1886487),
    313: (-32.6941798, -71.20176),
    312: (-32.6909181, -71.201165),
    311: (-32.750222, -71.160389),
    310: (-32.7419958, -71.1665269),
    309: (-32.7503708, -71.1604271),
    308: (-32.742028, -71.1665),
    307: (-32.763056, -71.1845),
    306: (-32.750277, -71.163907),
    305: (-32.710889, -71.241278),
    304: (-32.721444, -71.234167),
    303: (-32.689528, -71.2165),
    302: (-32.6960225, -71.2115997),
    301: (-32.6955166, -71.208724),
    300: (-32.685715, -71.215473),
    299: (-32.684783, -71.210869),
    298: (-32.6840769, -71.2286079),
    297: (-32.685133, -71.223217),
    269: (-32.687437, -71.209969),
    267: (-32.683842, -71.212023),
    261: (-33.548603, -71.487721),
    257: (-32.966667, -70.835472),
    242: (-33.006222, -70.886833),
    241: (-33.002444, -70.947417),
    223: (-33.000639, -71.003167),
    222: (-33.002861, -70.995111),
    219: (-33.091133, -70.753166),
    215: (-33.340028, -70.839972),
    213: (-33.272694, -70.694861),
    211: (-32.926917, -70.355472),
    209: (-33.167028, -70.886326),
    208: (-33.166971, -70.8872039),
    203: (-33.2739438, -70.8514864),
    199: (-32.768087, -70.992052),
    197: (-33.1155439, -70.8239401),
    196: (-33.1159641, -70.824824),
    192: (-33.244044, -70.681664),
    189: (-33.276182, -70.671453),
    183: (-33.106475, -70.793768),
    178: (-35.5268433, -71.701985),
    176: (-33.2170118, -70.7640911),
    175: (-33.2652098, -70.7250586),
    174: (-33.224127, -70.772071),
    172: (-33.21125, -70.658738),
    171: (-33.653369, -70.831969),
    169: (-33.2423, -70.693313),
    168: (-33.240798, -70.734343),
    167: (-33.246508, -70.734948),
    166: (-33.309871, -70.697893),
    165: (-33.305533, -70.689219),
    164: (-33.268053, -70.678775),
    161: (-33.293456, -70.699191),
    160: (-33.210595, -70.657842),
    158: (-33.023079, -70.686391),
    157: (-33.025832, -70.685635),
    154: (-33.225262, -70.772417),
    153: (-33.225435, -70.772395),
    152: (-33.071424, -70.719478),
    151: (-33.073067, -70.7177),
    150: (-33.174786, -70.690362),
    148: (-37.05378, -72.437578),
    147: (-33.421597, -70.784262),
    145: (-37.195614, -73.202642),
    144: (-33.632193, -70.714685),
    142: (-33.4640852, -70.8393559),
    141: (-31.77626, -70.978159),
    140: (-33.448013, -70.834865),
    139: (-33.457751, -70.8446723),
    135: (-33.427446, -70.822201),
    134: (-33.427446, -70.822201),
    133: (-33.4298495, -70.822877),
    132: (-33.076561, -70.771699),
    130: (-33.437121, -70.770068),
    129: (-33.092777, -70.727469),
    127: (-33.094392, -70.698918),
    122: (-33.358028, -70.830778),
    121: (-33.367229, -71.302691),
    120: (-33.08949, -70.72991),
    118: (-33.464639, -70.855417),
    117: (-33.4637383, -70.8553267),
    116: (-33.4706759, -70.8644792),
    115: (-33.4629167, -70.8552067),
    112: (-33.426937, -70.799965),
    111: (-33.734045, -70.763895),
    110: (-33.499937, -70.616457),
    109: (-33.499937, -70.616457),
    108: (-33.499937, -70.616457),
    102: (-36.63301, -72.1935167),
    101: (-40.5951383, -73.1032887),
    100: (-33.438227, -70.805888),
    99: (-33.437441, -70.787702),
    98: (-33.415896, -70.576656),
    97: (-41.447168, -72.953696),
    96: (-32.853665, -71.228907),
    94: (-35.091758, -71.321192),
    91: (-34.8775461, -71.1411382),
    89: (-33.381638, -70.592269),
    88: (-33.378165, -70.589793),
    87: (-33.377392, -70.596368),
    85: (-33.092662, -70.731019),
    83: (-38.726749, -72.596125),
    82: (-33.573549, -71.539515),
    80: (-33.427874, -70.778998),
    79: (-33.424673, -70.776482),
    76: (-34.106227, -70.447518),
    75: (-32.838673, -71.046394),
    74: (-34.108146, -70.447044),
    73: (-34.204379, -70.543976),
    71: (-33.283363, -70.642975),
    70: (-33.264848, -70.650676),
    69: (-33.277167, -70.630154),
    68: (-33.286625, -70.639999),
    66: (-33.413509, -70.576383),
    65: (-33.415493, -70.575682),
    64: (-33.384916, -70.586179),
    63: (-33.38861, -70.58901),
    62: (-33.507495, -70.604736),
    61: (-32.861894, -71.077483),
    60: (-32.848338, -71.082154),
    59: (-32.848297, -71.081817),
    58: (-32.863269, -71.076993),
    57: (-32.838306, -71.049545),
    56: (-32.840108, -71.0487),
    55: (-33.231236, -70.668991),
    54: (-33.224981, -70.664467),
    53: (-33.236942, -70.684166),
    52: (-33.23764, -70.689606),
    51: (-33.224018, -70.655167),
    50: (-33.237999, -70.675377),
    49: (-33.237622, -70.695634),
    48: (-33.232653, -70.670337),
    47: (-33.234382, -70.671339),
    46: (-33.242633, -70.683119),
    45: (-33.242426, -70.688208),
    44: (-33.239621, -70.69415),
    43: (-33.231221, -70.6376),
    42: (-33.243361, -70.670379),
    41: (-33.422426, -70.783969),
    40: (-33.427351, -70.770938),
    38: (-33.427348, -70.771908),
    29: (-34.2294, -70.885193),
    28: (-33.749015, -70.873931),
    27: (-35.06418, -71.295671),
    26: (-33.5325306, -70.7545842),
    25: (-33.480226, -70.694196),
    24: (-34.260017, -70.926789),
    23: (-33.3695512, -70.6937255),
    22: (-34.608125, -70.988034),
    21: (-34.458585, -70.955266),
    20: (-33.429539, -70.786132),
    19: (-34.342542, -70.853934),
    18: (-35.390493, -72.376777),
    17: (-41.5331539, -73.13008),
    16: (-30.152332, -71.22219),
    15: (-37.286615, -72.351514),
    14: (-33.792538, -70.753253),
    13: (-36.586847, -72.102788),
    12: (-33.227108, -70.816829),
    11: (-34.342735, -70.841097),
    10: (-33.901948, -71.410219),
    9: (-40.548909, -73.150981),
    8: (-33.332673, -70.695634),
    7: (-33.386757, -70.792773),
    6: (-41.41476, -72.9073),
    5: (-33.725935, -70.721214),
    4: (-33.2155393, -70.6752448),
    3: (-33.220972, -70.665093),
    2: (-20.748013, -70.192264),
    1: (-20.746193, -70.18151),
    327: (-32.661167, -71.199778),
    926: (-33.1869507, -70.6915307),
    1631: (-32.7497215, -71.1914787),
    1630: (-32.7474937, -71.1943855),
    1629: (-32.7493042, -71.193512),
    1406: (-33.11222, -70.74904),
    1405: (-33.12813, -70.76841),
    1623: (-41.456784, -72.963617),
    1025: (-32.9022560119629, -71.2011108398437)
}


id_punto = models.execute_kw(db, uid, password,
                'x_maintenance_location', 'search_read',
                [])

#Diccionario con la información la identificación de los puntos de monitoreo en odoo
dic_puntos = {}

for punto in id_punto:
    dic_puntos[punto['x_studio_asset_id_1']] = punto['id']

#print(dic_puntos)


for id in ubicaciones.keys():
    coordenadas = f'{ubicaciones[int(id)][0]}, {ubicaciones[int(id)][1]}'
    # print(coordenadas)
    try:
        update_loc = {
            'x_studio_char_field_M8G1K': coordenadas,
            'x_studio_ubicacin' : f'https://www.google.com/maps/place/{coordenadas}'
        }
        update = models.execute_kw(
            db, uid, password,
            'x_maintenance_location',
            'write',
            [
                [dic_puntos[str(id)]], 
                update_loc
            ],)
    except Exception as e:
        print(f"Error updating location for id {id}: {e}")


# for id_odoo, id in dic_puntos.items():
#     if id in list(ubicaciones.keys()):
#          print("s")
#         # coordenadas = f'{ubicaciones[int(id)][0]}, {ubicaciones[int(id)][1]}'
#         # try:
#         #     update_loc = {
#         #         'x_studio_char_field_M8G1K': coordenadas,
#         #         'x_studio_ubicacin' : f'https://www.google.com/maps/place/{coordenadas}'
#         #     }
#         #     update = models.execute_kw(
#         #         db, uid, password,
#         #         'x_maintenance_location',
#         #         'write',
#         #         [
#         #             [id_odoo], 
#         #             update_loc
#         #         ],)
#         # except Exception as e:
#         #     print(f"Error updating location for id {id_odoo}: {e}")
#     else:
#         print(f'No se encontró el id {id} en ubicaciones')

# print(dic_puntos)